In [60]:
import scipy.stats as st
from sympy.solvers import solve
from sympy import Symbol
from scipy.optimize import fsolve
import testjx
import numpy as np
import cvxopt
import cv2
import os 
import random
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from cvxopt import matrix
from cvxopt import solvers
import face_recognition
import sklearn.metrics as sm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
# from scikitplot import plotters as skplt

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from statsmodels.discrete.discrete_model import Logit, Probit, MNLogit
from pylab import mpl

import warningfrom sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib as mpl
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from collections import Counter
import cv2
import os
import glob
import skimage
import numpy as np
import pandas as pd
import seaborn as sn
import preprocessing
from tqdm import tqdm
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
from skimage.transform import resize
from collections import Counter
import imblearn
from sklearn.metrics import balanced_accuracy_score
sn.set()
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC # SVC
from sklearn import metrics
from sklearn.utils import shuffle
from xgboost import XGBClassifier # XGBClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,f1_score,recall_score,cohen_kappa_score,precision_score
from sklearn.utils import compute_class_weight
from sklearn.preprocessing import MinMaxScaler,LabelBinarizer
from sklearn.ensemble import AdaBoostClassifier # AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier # KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier # RandomForestClassifier
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.applications.vgg19 import VGG19 # VGG19
from tensorflow.keras.applications.resnet50 import ResNet50 # ResNet50
from tensorflow.keras.applications.xception import Xception # Xception
from tensorflow.keras.applications.mobilenet import MobileNet # MobileNet
from tensorflow.keras.applications.nasnet import NASNetMobile # NASNetMobile
from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet169
from tensorflow.keras.applications.densenet import DenseNet121 # DenseNet121
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3 # InceptionV3
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## Feature extraction

In [2]:
files = os.listdir('/Users/lvjingzhe/Downloads/celebA/train/')
train = []
    
for i in range(20000):
# for i in range(20000):
    img_ = cv2.imread('/Users/lvjingzhe/Downloads/celebA/train/%d.jpg' %(i))

    train.append(img_)

# files = os.listdir('/Users/lvjingzhe/Downloads/celebA/test/')
# test = []
    
# for i in range(2000,2200):
#     img_ = cv2.imread('/Users/lvjingzhe/Downloads/celebA/test/%d.jpg' %(i))

#     test.append(img_)

In [4]:
train_image=np.array(train)
train_label=np.concatenate((np.array([0]*19000),np.array([1]*1000)))

In [5]:
''' Feature extraction---VGG19 network'''


import time
start=time.time()
base_model= VGG19( weights='imagenet', include_top=False,input_shape=(64,64,3))
x = base_model.output
# x = Dropout(0.2)(x)
predictions = Flatten()(x)

model_feat = Model(inputs=base_model.input,outputs=predictions)

train_features = model_feat.predict(train_image/255)
# test_features=model_feat.predict(test_image/255)
end=time.time()
print('running time:{}'.format(end-start))

W0127 09:47:16.641349 140736062219136 deprecation.py:506] From /Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


running time:255.947163105011


In [6]:
''' Shape of input length and feature dimension after VGG19'''
train_features.shape


(20000, 2048)

In [6]:
'''Divide into two sets: training set(x_tr,y_tr), and testing set (x_te,y_te)'''
x_tr, x_te, y_tr, y_te = train_test_split(train_features,train_label,test_size = 0.2,
                                                  shuffle = True,
                                                  random_state = 42)

In [7]:

X_test=np.c_[x_te,np.ones(x_te.shape[0])]

In [8]:
'''Split the training set into 20 groups, one of which contains the toral glassed ones.
This step is useful in max-mean loss method
 (x_tr,y_tr) becomes (pre_XX,pre_YY) after "Split", which is the input of max-mean loss method.'''
def split(x):
    n=y_tr[y_tr==1].shape[0]
    m=int((X_train.shape[0]-n)/19)
    indice=[k*m for k in range(1,19)]
    indice.append(X_train.shape[0]-n)
    x_c=np.concatenate((x[y_tr==0],x[y_tr==1]))
    return np.array(np.split(x_c,indice))

X_train=np.c_[x_tr,np.ones(x_tr.shape[0])]
n=y_tr[y_tr==1].shape[0]
XX=np.concatenate((X_train[y_tr==0],X_train[y_tr==1]))
YY=np.concatenate((np.array([0]*(X_train.shape[0]-n)),np.array([1]*n)))
m=int((X_train.shape[0]-n)/19)
indice=[k*m for k in range(1,19)]
indice.append(X_train.shape[0]-n)

pre_XX=np.array(np.split(XX,indice))
pre_YY=np.array(np.split(YY,indice))

In [10]:
''' Split training set into 5 cross-validation sets'''
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=0)

## Traditional classifiers：LR、CS_LR、AUSTBOOST、BAGGING

In [82]:
'''model_in symbols the result of traditional logistic regression'''

import time
start_i=time.time()
model_in=LogisticRegression()

model_in.fit(x_tr,y_tr)
end_i=time.time()
y_train_proba=model_in.predict_proba(x_tr)
y_train_label=model_in.predict(x_tr)
y_test_proba=model_in.predict_proba(x_te)

y_test_label=model_in.predict(x_te)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print("Running time:%.2f s"%(end_i-start_i))
from sklearn.metrics import accuracy_score

print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
# print(classification_report(y_tr,y_train_label))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      3812
           1       0.80      0.69      0.74       188

    accuracy                           0.98      4000
   macro avg       0.89      0.84      0.86      4000
weighted avg       0.98      0.98      0.98      4000

Running time:3.21 s
Balanced-Accuracy on testing set：84.14%
Recall on testing set：69.15%
F-measure on testing set：70.12%


In [147]:
##SVM
from sklearn.metrics import fbeta_score
import time
start=time.time()
model_svc=SVC(probability=True)

model_svc.fit(x_tr,y_tr)
end=time.time()
y_train_proba=model_svc.predict_proba(x_tr)
y_train_label=model_svc.predict(x_tr)
y_test_proba=model_svc.predict_proba(x_te)

y_test_label=model_svc.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
# from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print("Running time:%.2f s"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of SVM：{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3812
           1       0.98      0.48      0.65       188

    accuracy                           0.98      4000
   macro avg       0.98      0.74      0.82      4000
weighted avg       0.98      0.98      0.97      4000

Running time:166.58 s
Balanced-Accuracy on testing set：74.18%
Recall on testing set：48.40%
F-measure on testing set：51.08%
Score of SVM：0.573


In [129]:
'''(X_resampled, y_resampled) is the training set after randomly resampling method'''
from imblearn.over_sampling import RandomOverSampler,SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)

X_resampled, y_resampled = rus.fit_resample(x_tr,y_tr)

In [145]:
##RUS-SVM
import time
start=time.time()
model_svc_r=SVC(gamma='scale',probability=True)
# model_in.fit(pt.fit_transform(train_image_),train_label_)
model_svc_r.fit(X_resampled, y_resampled)
end=time.time()
y_train_proba=model_svc_r.predict_proba(x_tr)
y_train_label=model_svc_r.predict(x_tr)
y_test_proba=model_svc_r.predict_proba(x_te)
# print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=model_svc_r.predict(x_te)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print(classification_report(y_te,y_test_label))
print("Running time:%.2f s"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of RUS-SVM:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))




              precision    recall  f1-score   support

           0       0.99      0.88      0.93      3812
           1       0.27      0.90      0.42       188

    accuracy                           0.88      4000
   macro avg       0.63      0.89      0.68      4000
weighted avg       0.96      0.88      0.91      4000

Running time:6.64 s
Balanced-Accuracy on testing set：89.31%
Recall on testing set：90.43%
F-measure on testing set：73.03%
Score of RUS-SVM:0.836


In [146]:
##SVM+cost-learning
import time
start=time.time()
model_svc3=SVC(class_weight='balanced',probability=True)
# model_in.fit(pt.fit_transform(train_image_),train_label_)
# model_svc3.fit(X_resampled_smote,y_resampled_smote)
model_svc3.fit(x_tr,y_tr)
end=time.time()
y_train_proba=model_svc3.predict_proba(x_tr)
y_train_label=model_svc3.predict(x_tr)
y_test_proba=model_svc3.predict_proba(x_te)

y_test_label=model_svc3.predict(x_te)

bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :CS-SVM')
print("Running time:%.2f s"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



Classifier :CS-SVM
Running time:487.11 s
Balanced-Accuracy on testing set：90.13%
Recall on testing set：86.17%
F-measure on testing set：77.65%
Score :0.838


In [134]:
##RUSBoost
from imblearn.ensemble import RUSBoostClassifier
rusb = RUSBoostClassifier(random_state=0)
start=time.time()
rusb.fit(x_tr,y_tr)
end=time.time()
y_train_proba=rusb.predict_proba(x_tr)
y_train_label=rusb.predict(x_tr)
y_test_proba=rusb.predict_proba(x_te)
# print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=rusb.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print('Classifier :RUS-Boost')
print("Running time:%.2f s"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))




              precision    recall  f1-score   support

           0       0.99      0.89      0.94      3812
           1       0.26      0.78      0.39       188

    accuracy                           0.88      4000
   macro avg       0.62      0.83      0.66      4000
weighted avg       0.95      0.88      0.91      4000

Classifier :RUS-Boost
Running time:8.22 s
Balanced-Accuracy on testing set：83.32%
Recall on testing set：77.66%
F-measure on testing set：64.26%
Score:0.745


In [140]:
##MLP
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(random_state=0, max_iter=200,hidden_layer_sizes=(50,))
start=time.time()


mlp.fit(x_tr,y_tr)
end=time.time()
y_train_proba=mlp.predict_proba(x_tr)
y_train_label=mlp.predict(x_tr)
y_test_proba=mlp.predict_proba(x_te)
# print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=mlp.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :MLP')
print("Running time:%.2f s"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



Classifier :MLP
Running time:24.46 s
Balanced-Accuracy on testing set：84.09%
Recall on testing set：69.15%
F-measure on testing set：69.96%
Score:0.737


In [138]:
from imblearn.over_sampling import SMOTE

X_resampled_smote,y_resampled_smote  = SMOTE().fit_resample(x_tr, y_tr)

In [141]:
##MLP+SMOTE
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(random_state=0, max_iter=200,hidden_layer_sizes=(50,))
start=time.time()


mlp.fit(X_resampled_smote,y_resampled_smote)
end=time.time()
y_train_proba=mlp.predict_proba(x_tr)
y_train_label=mlp.predict(x_tr)
y_test_proba=mlp.predict_proba(x_te)

y_test_label=mlp.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :SMOTE-MLP')
print("Running time:%.2f s"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3812
           1       0.73      0.73      0.73       188

    accuracy                           0.97      4000
   macro avg       0.86      0.86      0.86      4000
weighted avg       0.97      0.97      0.97      4000

Classifier :SMOTE-MLP
Running time:16.76 s
Balanced-Accuracy on testing set：86.05%
Recall on testing set：73.40%
F-measure on testing set：73.40%
Score:0.769


In [137]:
##MLP+RUS
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(random_state=10, max_iter=200,hidden_layer_sizes=(50,))
start=time.time()


mlp.fit(X_resampled,y_resampled)
end=time.time()
y_train_proba=mlp.predict_proba(x_tr)
y_train_label=mlp.predict(x_tr)
y_test_proba=mlp.predict_proba(x_te)

y_test_label=mlp.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :RUS-MLP')
print("Running time:%.2f s"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



Classifier :RUS-MLP
Running time:3.10 s
Balanced-Accuracy on testing set：90.13%
Recall on testing set：92.02%
F-measure on testing set：74.28%
Score :0.849


In [142]:
##CART+cost-learing
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(class_weight='balanced',random_state=0)
start=time.time()


clf.fit(x_tr,y_tr)
end=time.time()
y_train_proba=clf.predict_proba(x_tr)
y_train_label=clf.predict(x_tr)
y_test_proba=clf.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=clf.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :CS-Cart')
print("Running time:%.2f s"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of CS-Cart:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.6801505882878257
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      3812
           1       0.37      0.39      0.38       188

    accuracy                           0.94      4000
   macro avg       0.67      0.68      0.67      4000
weighted avg       0.94      0.94      0.94      4000

Classifier :CS-Cart
Running time:15.60 s
Balanced-Accuracy on testing set：68.02%
Recall on testing set：39.36%
F-measure on testing set：39.08%
Score of CS-Cart:0.484


In [143]:
##CART+SMOTE
clf = DecisionTreeClassifier(random_state=0)
start=time.time()


clf.fit(X_resampled_smote,y_resampled_smote)
end=time.time()
y_train_proba=clf.predict_proba(x_tr)
y_train_label=clf.predict(x_tr)
y_test_proba=clf.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=clf.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :SMOTE-Cart')
print("Running time:%.2f s"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.7087277578084883
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      3812
           1       0.31      0.47      0.38       188

    accuracy                           0.93      4000
   macro avg       0.64      0.71      0.67      4000
weighted avg       0.94      0.93      0.93      4000

Classifier :SMOTE-Cart
Running time:110.93 s
Balanced-Accuracy on testing set：70.87%
Recall on testing set：46.81%
F-measure on testing set：44.52%
Score :0.535


In [49]:
##CART+RUS
clf = DecisionTreeClassifier(random_state=0)
start=time.time()


clf.fit(X_resampled,y_resampled)
end=time.time()
y_train_proba=clf.predict_proba(x_tr)
y_train_label=clf.predict(x_tr)
y_test_proba=clf.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=clf.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print('Classifier :RUS-Cart')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score of RUS-Cart:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.7916210845928869
              precision    recall  f1-score   support

           0       0.99      0.77      0.87      3812
           1       0.15      0.81      0.25       188

    accuracy                           0.77      4000
   macro avg       0.57      0.79      0.56      4000
weighted avg       0.95      0.77      0.84      4000

Classifier :RUS-Cart
Running time:1.25秒
Balanced-Accuracy on testing set：79.16%
Recall on testing set：81.38%
F-measure on testing set：55.53%
Score of RUS-Cart:0.721


In [1005]:
(0.555+0.792+0.81)*0.3+0.1/1.22

0.7290672131147541

In [45]:
from imbalanced_ensemble.ensemble import SMOTEBoostClassifier
smb = SMOTEBoostClassifier(random_state=0)


In [624]:
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(x_tr,y_tr)

In [52]:
##SMOTEBoost
start=time.time()
smb.fit(x_tr,y_tr)
end=time.time()
y_train_proba=smb.predict_proba(x_tr)
y_train_label=smb.predict(x_tr)
y_test_proba=smb.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=smb.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print('Classifier :SMOTE-Boost')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.918534136322029
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      3812
           1       0.46      0.63      0.53       188

    accuracy                           0.95      4000
   macro avg       0.72      0.80      0.75      4000
weighted avg       0.96      0.95      0.95      4000

Classifier :SMOTE-Boost
Running time:234.80秒
Balanced-Accuracy on testing set：79.81%
Recall on testing set：63.30%
F-measure on testing set：60.91%
Score :0.673


In [50]:
adb=AdaBoostClassifier(random_state=0)
start=time.time()
adb.fit(X_resampled, y_resampled )
end=time.time()
y_train_proba=adb.predict_proba(x_tr)
y_train_label=adb.predict(x_tr)
y_test_proba=adb.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=adb.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print('Classifier :RUS-Boost')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



AUC:0.9448515047665825
              precision    recall  f1-score   support

           0       0.99      0.85      0.92      3812
           1       0.23      0.90      0.37       188

    accuracy                           0.86      4000
   macro avg       0.61      0.88      0.64      4000
weighted avg       0.96      0.86      0.89      4000

Classifier :RUS-Boost
Running time:6.59秒
Balanced-Accuracy on testing set：87.67%
Recall on testing set：89.89%
F-measure on testing set：69.38%
Score :0.816


In [51]:
##ADABoost
adb=AdaBoostClassifier(random_state=0)
start=time.time()
adb.fit(x_tr,y_tr)
end=time.time()
y_train_proba=adb.predict_proba(x_tr)
y_train_label=adb.predict(x_tr)
y_test_proba=adb.predict_proba(x_te)

y_test_label=adb.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))

print('Classifier :Adaboost')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3812
           1       0.73      0.55      0.63       188

    accuracy                           0.97      4000
   macro avg       0.85      0.77      0.81      4000
weighted avg       0.97      0.97      0.97      4000

Classifier :Adaboost
Running time:81.01秒
Balanced-Accuracy on testing set：76.90%
Recall on testing set：54.79%
F-measure on testing set：56.25%
Score:0.62


In [54]:
##BAgging
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier(
                                random_state=0)


start=time.time()


bag.fit(x_tr,y_tr)
end=time.time()
y_train_proba=bag.predict_proba(x_tr)
y_train_label=bag.predict(x_tr)
y_test_proba=bag.predict_proba(x_te)
print('AUC:{}'.format(roc_auc_score(y_te,y_test_proba[:,-1])))
y_test_label=bag.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :Bagging')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))




AUC:0.872678104976446
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3812
           1       0.84      0.44      0.58       188

    accuracy                           0.97      4000
   macro avg       0.91      0.72      0.78      4000
weighted avg       0.97      0.97      0.97      4000

Classifier :Bagging
Running time:285.68秒
Balanced-Accuracy on testing set：71.86%
Recall on testing set：44.15%
F-measure on testing set：46.43%
Score:0.536


In [56]:
##Bagging+RUS
bag = BaggingClassifier(
                                random_state=0)


start=time.time()


bag.fit(X_resampled,y_resampled)
end=time.time()
y_train_proba=bag.predict_proba(x_tr)
y_train_label=bag.predict(x_tr)
y_test_proba=bag.predict_proba(x_te)

y_test_label=bag.predict(x_te)
# y_test_proba=model_in.predict_proba(image_test)
# y_test_label=model_in.predict(image_test)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
bacc=balanced_accuracy_score(y_te,y_test_label)
fbeta=fbeta_score(y_te,y_test_label,beta=2.94)
recall=recall_score(y_te,y_test_label)
print('Classifier :RUS-Bag')
print("Running time:%.2f秒"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



              precision    recall  f1-score   support

           0       0.99      0.86      0.92      3812
           1       0.23      0.85      0.36       188

    accuracy                           0.86      4000
   macro avg       0.61      0.85      0.64      4000
weighted avg       0.96      0.86      0.90      4000

Classifier :RUS-Bag
Running time:8.17秒
Balanced-Accuracy on testing set：85.40%
Recall on testing set：84.57%
F-measure on testing set：66.41%
Score:0.781


In [821]:
split(err)[0].shape

(799,)

In [12]:
''' Class solution_XX() is the algorithm based on Xiaohua,Xuan[2019], the improvement is that
we add a parameter of "penalty".'''
def split(x):
    n=y_tr[y_tr==1].shape[0]
    m=int((X_train.shape[0]-n)/19)
    indice=[k*m for k in range(1,19)]
    indice.append(X_train.shape[0]-n)
    x_c=np.concatenate((x[y_tr==0],x[y_tr==1]))
    return np.array(np.split(x_c,indice))

class solution_XX:
    
    
    def __init__(self,penalty = None,Lambda = 0.03,a = 0.5,epochs = 200):
        self.W = None
        self.penalty = penalty
        self.Lambda = Lambda
        self.a = a
        self.epochs =epochs
        self.sigmoid = lambda x:1/(1 + np.exp(-x))
        
#     err=(y_tr-s.sigmoid(np.dot(X_train,w)))**2
#     err[index]=2.6*err[index]

    def f_XX(self,X,Y):
        if self.penalty=='l1':f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 )+self.Lambda*np.sum(np.abs(self.W)) for x,y in zip(X,Y)])
        elif self.penalty=='l2':f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 )+self.Lambda*np.sum(self.W**2) for x,y in zip(X,Y)])#pre_Xtrain,pre_Ytrain
        else:f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 ) for x,y in zip(X,Y)])

        return f         
    def Gf_XX(self,X,Y):#To compute the Derivative matrix, the shape of which is N*2
        if self.penalty=='l1':d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W))))+self.Lambda*np.sign(self.W )for x,y in zip(X,Y)])
        elif self.penalty=='l2':
            d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W))))+2*self.Lambda*self.W for x,y in zip(X,Y)])
#     return d.reshape(20,12289)
        else:d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W)))) for x,y in zip(X,Y)])
        return d
    def direction_XX(self,X,Y):
        gra=self.Gf_XX(X,Y)
        p=matrix(gra.dot(gra.T),tc='d')
        q=matrix(-self.f_XX(X,Y),tc='d')
        G=matrix(np.diag(np.array([-1]*20)),tc='d')#N=20
        h=matrix(np.array([[0]]*20),tc='d')
        A=matrix([[1.0]]*20)
        b=matrix([1.0])
        solvers.options['show_progress'] = False
        sol = solvers.qp(p,q,G,h,A,b)
        t=np.array(sol['x'])
        d= -(gra.T.dot(t))
        return d.reshape((X_train.shape[-1],))

    def fit(self,X,Y):
        
        call=[]
        pre=[]
        loss=[]
        testloss=[]
        np.random.seed(1324)
        self.W=np.random.random((X_train.shape[-1],))*2-1
#         self.W=w_2
        n=y_te[y_te==1.].shape[0]
        for k in range(200):
    #     while np.linalg.norm(d)//10**(-8) >= 10:
            d=self.direction_XX(X,Y)
#             print(np.linalg.norm(d))
            if np.linalg.norm(d)//10**(-7) < 25:
                break
            sigma=0.8
            f_1=np.max(self.f_XX(X,Y))
            w=self.W
            self.W=d*sigma+w
            while np.max(self.f_XX(X,Y))>np.max(f_1):
                sigma=sigma*0.8
                self.W=d*sigma+w
            self.W=d*sigma+w

            
    #         output=output.reshape(4000,)
    #         pt=max(output[output==1.].shape[0],1)
    #         m=0
    #         for j in range(4000):
    #             if output[j]==test_label[j]==1:
    #                 m+=1


    # # b1=np.random.random((1,10))*2-1


            
    #         call.append(m/n)
    #         pre.append(m/pt)
    #         loss.append(max(f_X(w)))
    #         testloss.append(ff_X(w))
        return self.W

In [13]:

s_2=solution_XX(penalty='l2')
start_x=time.time()
w_2 = s_2.fit(pre_XX,pre_YY)'''w_2 is the estimator obtained by max-mean loss method+L2 regulation'''
end_x=time.time()
y_pred=s_2.sigmoid(X_test.dot(w_2))

y_train_pred=s_2.sigmoid(X_train.dot(w_2))
y_train_pred[y_train_pred>=0.5]=1
y_train_pred[y_train_pred<0.5]=0
y_pred[y_pred>=0.5]=1
y_pred[y_pred<0.5]=0

bacc=balanced_accuracy_score(y_te,y_pred)
fbeta=fbeta_score(y_te,y_pred,beta=2.94)
recall=recall_score(y_te,y_pred)
print('Classifier :L2 max-mean loss')
print("Running time:%.2f s"%(end_x-start_x))

print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_pred)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_pred)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_pred,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_x-start_x)))



Classifier :L2 max-mean loss
Running time:20.98 s
Balanced-Accuracy on testing set：89.45%
Recall on testing set：85.64%
F-measure on testing set：76.00%
Score:0.829


In [68]:
s=solution_XX()
start=time.time()
w = s.fit(pre_XX,pre_YY)
end=time.time()
y_pred=s.sigmoid(X_test.dot(w))

y_train_pred=s.sigmoid(X_train.dot(w))
y_train_pred[y_train_pred>=0.5]=1
y_train_pred[y_train_pred<0.5]=0
y_pred[y_pred>=0.5]=1
y_pred[y_pred<0.5]=0
# # C_x.append(precision_score(y_pred,train_label_[test_index]))
bacc=balanced_accuracy_score(y_te,y_pred)
fbeta=fbeta_score(y_te,y_pred,beta=2.94)
recall=recall_score(y_te,y_pred)
print('Classifier : max-mean loss')
print("Running time:%.2f"%(end-start))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_pred)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_pred)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_pred,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end-start)))



Classifier : max-mean loss
Running time:13.82
Balanced-Accuracy on testing set：88.20%
Recall on testing set：83.51%
F-measure on testing set：73.75%
Score:0.811


## Mean_uncertain method

In [14]:
def sigmoid_(x,u):
    return 1.0/(1.0+np.exp(-x-u))

In [83]:
pre_in=(np.dot(x_tr,model_in.coef_.T)+model_in.intercept_).reshape(x_tr.shape[0],)
## pre_in is the product of training set and traditional estimator obtained by LR.


In [16]:
def meanuncertainty(x,n):
    r=[]
    for i in range(0,len(x)+1-n,n//5):
        r.append(np.mean(x[i:i+n]))
    return min(r),max(r)

In [17]:
def equa_xh(x,n):
    pre_in=np.dot(X_train,w_2).reshape(x_tr.shape[0],)
    ini_err=y_tr-sigmoid_(pre_in,x)##ini_err is the predicted err of training set, based on max-mean loss.
    err_w=shuffle(np.concatenate((np.random.choice(ini_err[y_tr==0],ini_err[y_tr==1].shape[0]),ini_err[y_tr==1])))
    for k in range(ini_err.shape[0]):
    
        if y_tr[k]==1:
            ini_err[k]=10*ini_err[k]

    
    return meanuncertainty(ini_err,n)[1]## the return is the max-mean err


In [84]:
def equa_m(x,n):
    
    ini_err=y_tr-sigmoid_(pre_in,x)##ini_err is the predicted err of training set, based on LR

    for k in range(ini_err.shape[0]):
    
        if y_tr[k]==1:
            ini_err[k]=10*ini_err[k]

    
    return meanuncertainty(ini_err,n)[1]


In [123]:
''' each meamber of meanxh represents the upper mean of max-mean loss with respect to fixed window size N. '''
meanxh_1=np.array([fsolve(lambda x:equa_xh(x,n),0.5 ) for n in range(10,1000,10)])
# meanxh_2=np.array([fsolve(lambda x:equa_xh(x,n),0.5 ) for n in range(1000,10000,100)])

In [44]:
pd.DataFrame(meanxh_1,index=np.array(range(10,1000,10))).to_csv('glass_upper_mean.csv',index=False)

In [72]:
'''each meamber of mean represents the upper mean of LR with respect to fixed window size N.'''
start=time.time()
mean_1=np.array([fsolve(lambda x:equa_m(x,n),0.5 ) for n in range(10,1000,10)])
end=time.time()
print(end-start)
# mean_2=np.array([fsolve(lambda x:equa_m(x,n),0.5 ) for n in range(1000,10000,100)])

7.375096082687378


In [282]:

mean_lr=np.concatenate((mean_1,mean_2))[:-20].reshape(-1)


In [966]:
12800/650

19.692307692307693

## Choice optimal upper mean on  Cross-validation set

In [107]:

"""We use 'evaluate_modelxh' to obtain the average performance in 5-fold cross-validation set, for fixed upper mean 'm' """

metrics_names=['Balanced_acc','F2_score','Recall','Precision']
def evaluate_modelxh(X,y,m,model=solution_XX()):
    
    bacc,f2,rec,pre=list(),list(),list(),list()
   

    for train_index,test_index in cv.split(X,y):
        
#         N=X[train_index].shape[0]
#         n=y[train_index][y[train_index]==1].shape[0]
#         c_1=np.concatenate((X[train_index][y[train_index]==0],X[train_index][y[train_index]==1]))
#         l_1=np.concatenate((np.array([0]*(N-n)),np.array([1]*n)))
#         M=int((N-n)/19)/
#         indice=[k*M for k in range(1,19)]
#         indice.append(N-n)
#         pre_cv=np.array(np.split(c_1,indice))
#         pre_y=np.array(np.split(l_1,indice))
#         start=time.time()
#         w= model.fit(pre_cv,pre_y)
#         end=time.time()
#         print(end-start)
#         w_x.append(w)
        prob_y=sigmoid_(X_train[test_index].dot(w_2),m)
        
        prdict_y=np.round(prob_y)
        rec.append(recall_score(y[test_index],prdict_y))
        pre.append(precision_score(y[test_index],prdict_y))

        bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
        f2.append(fbeta_score(y[test_index],prdict_y,beta=2.94))
 
    return bacc,f2,rec,pre
    
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
           
               

In [124]:

all_resultsxh = []

for k in meanxh_1:
    result=np.mean(evaluate_modelxh(X_train,y_tr,k),axis=1)
    metric_res = {'upper_mean': k}
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_resultsxh.append(metric_res)

In [125]:
'''We compare the performance of different upper mean obtained above, the result is saved in .csv file.'''

all_resultsxh_w = []

for k in meanxh_1:
    result=np.mean(evaluate_modelxh(X_train,y_tr,k),axis=1)
    metric_res = {'upper_mean': k}
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_resultsxh_w.append(metric_res)

In [127]:
pd.DataFrame(all_resultsxh_w).to_csv('/Users/lvjingzhe/Desktop/璇/modified_althogram/code/CV_select_mu_MM_gla_simple.csv')

In [73]:

metrics_names=['Balanced_acc','F2_score','AUC','Recall','Precision']
def evaluate_model(X,y,m,model=LogisticRegression()):
    
    bacc,f2,auc,rec,pre=list(),list(),list(),list(),list()
   
#     cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=1, random_state=1)
    for train_index,test_index in cv.split(X,y):
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
#                 model.fit(X[train_index],y[train_index])
                prob_y=sigmoid_((np.dot(X[test_index],model_in.coef_.T)+model_in.intercept_),m)
                auc.append(roc_auc_score(y[test_index],prob_y))
                
                prdict_y=np.round(prob_y)
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=2.94))
 
    return bacc,f2,auc,rec,pre

In [37]:

all_results = []
c=1
# for k in mean_lr:
strat=time.time()
for k in mean_1.reshape(-1):
    result=np.mean(evaluate_model(x_tr,y_tr,k),axis=1)
    metric_res = {'window':10*c,'upper_mean': k}
    c+=1
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_results.append(metric_res)
end=time.time()
end-start

3187.276778936386

In [286]:

pd.DataFrame(all_results).to_csv('CV_which_mu_to_set_inLRBIAS.csv')


## Volatility-uncertain method


In [100]:
'''F_u and F_L compute the maximal and minimal probability of Y=1 '''
 
def F_u(x,arr):
    p=[]
    for c in arr:
        
        if c <0:
            p.append(2*x[1]*st.norm.cdf(c/x[1])/(x[0]+x[1]))
        else:
            p.append(1-2*x[0]*st.norm.cdf(-c/x[0])/(x[0]+x[1]))
    return np.array(p)
def F_L(x,arr):
    p=[]
    for c in arr:
        
        if c <0:
            p.append(2*x[0]*st.norm.cdf(c/x[0])/(x[0]+x[1]))
        else:
            p.append(1-2*x[1]*st.norm.cdf(-c/x[1])/(x[0]+x[1]))
    return np.array(p)

In [101]:
'''equa_v returns the maximal and minimal err obtained by LR for fixed window size n'''

def equa_v(x,n):
 
    
    err_u=y_tr-F_u(x,pre_in)
    err_L=y_tr-F_L(x,pre_in)
#     for k in range(y_tr.shape[0]):
    
#         if y_tr[k]==1:
#             err_u[k]=2*err_u[k]
#             err_L[k]=19*err_L[k]
    
    return np.array([meanuncertainty(err_u,n)[1],meanuncertainty(err_L,n)[0]])

In [104]:
'''equa_vxh returns the maximal and minimal err obtained by max-mean loss for fixed window size n'''

def equa_vxh(x,n):
    pre_in=np.dot(X_train,w_2).reshape(x_tr.shape[0],)
    err_u=y_tr-F_u(x,pre_in)
    err_L=y_tr-F_L(x,pre_in)
#     for k in range(y_tr.shape[0]):
    
#         if y_tr[k]==1:
#             err_u[k]=2*err_u[k]
#             err_L[k]=19*err_L[k]
    
    return np.array([meanuncertainty(err_u,n)[1],meanuncertainty(err_L,n)[0]])

In [106]:
fsolve(lambda x:equa_v(x,500),[0.5,1.5] )

array([0.93177559, 2.08727278])

In [1094]:

var_2=np.array([
    for n in range(500,1000,50)])
var_3=np.array([fsolve(lambda x:equa_vxh(x,n),[0.5,1.5] ) for n in range(1000,6000,100)])



In [1096]:
'''var_xh is the volatility uncertainty with respect to different windowsize n.'''

var_xh=np.concatenate((var_2,var_3))
var_xhmean=pd.DataFrame(var_xh,index=np.hstack((
                                           np.array(range(500,1000,50)),np.array(range(1000,6000,100)))))
var_xhmean.to_csv('var_xhmean_celeb.csv')

## Choice optimal window size on  Cross-validation set

In [82]:
metrics_names=['Balanced_acc','F2_score','Recall','Precision']
def evaluate_varxh(X,y,m,model=solution_XX(penalty='l2')):
    
    bacc,f2,rec,pre=list(),list(),list(),list()
   

    for train_index,test_index in cv.split(X,y):
        
        N=X[train_index].shape[0]
        n=y[train_index][y[train_index]==1].shape[0]
        c_1=np.concatenate((X[train_index][y[train_index]==0],X[train_index][y[train_index]==1]))
        l_1=np.concatenate((np.array([0]*(N-n)),np.array([1]*n)))
        M=int((N-n)/19)
        indice=[k*M for k in range(1,19)]
        indice.append(N-n)
        pre_cv=np.array(np.split(c_1,indice))
        pre_y=np.array(np.split(l_1,indice))
        w= model.fit(pre_cv,pre_y)
#         w_x.append(w)
        prob_y=F_u(m,X_train[test_index].dot(w))
        
        prdict_y=np.round(prob_y)
        rec.append(recall_score(y[test_index],prdict_y))
        pre.append(precision_score(y[test_index],prdict_y))

        bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
        f2.append(fbeta_score(y[test_index],prdict_y,beta=2.94))
 
    return bacc,f2,rec,pre

In [85]:
for train_index,test_index in cv.split(X_train,y_tr):
        
        N=X_train[train_index].shape[0]
        n=y_tr[train_index][y_tr[train_index]==1].shape[0]
        c_1=np.concatenate((X_train[train_index][y_tr[train_index]==0],X_train[train_index][y_tr[train_index]==1]))
        l_1=np.concatenate((np.array([0]*(N-n)),np.array([1]*n)))
        M=int((N-n)/19)
        indice=[k*M for k in range(1,19)]
        indice.append(N-n)
        pre_cv=np.array(np.split(c_1,indice))
        pre_y=np.array(np.split(l_1,indice))

In [1104]:

all_resultsxhvar = []

for k in var_xh:
    result=np.mean(evaluate_varxh(X_train,y_tr,k),axis=1)
    metric_res = {'lower-var': k[0],'upper-var':k[1]}
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_resultsxhvar.append(metric_res)

In [1105]:
pd.DataFrame(all_resultsxhvar).to_csv('CV_which_sigma_to_set_inxhBIAS.csv')

In [ ]:
var_1=np.array([fsolve(lambda x:equa_v(x,n),[0.5,1.5] ) for n in range(50,110,10)])
var_2=np.array([fsolve(lambda x:equa_v(x,n),[0.5,1.5] ) for n in range(110,1000,50)])
var_3=np.array([fsolve(lambda x:equa_v(x,n),[0.5,1.5] ) for n in range(1000,6000,100)])


                

In [566]:
var=np.concatenate((var_1,var_2,var_3))

In [571]:
var_mean=pd.DataFrame(var,index=np.hstack((np.array(range(50,110,10)),
                                           np.array(range(110,1000,50)),np.array(range(1000,6000,100)))))

In [573]:
var_mean.to_csv('var_mean_celeb.csv')

In [576]:
def var_evaluate_model(X,y,m,model=LogisticRegression()):
    
    bacc,f2,auc,rec,pre=list(),list(),list(),list(),list()
   
#   
    for train_index,test_index in cv.split(X,y):
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
                model.fit(X[train_index],y[train_index])
                prob_y=F_u(m,(np.dot(X[test_index],model_in.coef_.T)+model_in.intercept_).reshape(X[test_index].shape[0],))
        
                auc.append(roc_auc_score(y[test_index],prob_y))
                
                prdict_y=np.round(prob_y)
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=2))
 
    return bacc,f2,auc,rec,pre

In [585]:

all_results_var = []

for k in var:
    result=np.mean(var_evaluate_model(x_tr,y_tr,k),axis=1)
    metric_res = {'lower_var':k[0],'upper_var': k[1]}
    for name, value in zip(metrics_names, result):
#             print(name, ': ', value)
            metric_res[name] = value
       

    all_results_var.append(metric_res)

In [587]:
pd.DataFrame(all_results_var).to_csv('CV_which_sigma_to_set_inLRBIAS.csv')

## testing performance

In [86]:
'''volatility-bias max-mean: w_2 is the estimator obtained by max-mean loss,[0.075,0.487] is the volatility uncertainty
obtained by cross-validition analysis
'''
y_preX_v=F_u([0.075,0.487],np.dot(X_test,w_2)).reshape(x_te.shape[0],)

y_preX_v[y_preX_v<0.5]=0
y_preX_v[y_preX_v>=0.5]=1
print(classification_report(y_te,y_preX_v))
bacc=balanced_accuracy_score(y_te,y_preX_v)
fbeta=fbeta_score(y_te,y_preX_v,beta=2.94)
recall=recall_score(y_te,y_preX_v)
print('Classifier : volatility-unceratin max-mean ')
print("Running time:%.2f"%(end_x-start_x))

print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_preX_v)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_preX_v)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_preX_v,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_x-start_x)))



              precision    recall  f1-score   support

           0       1.00      0.90      0.95      3812
           1       0.32      0.92      0.47       188

    accuracy                           0.90      4000
   macro avg       0.66      0.91      0.71      4000
weighted avg       0.96      0.90      0.92      4000

Classifier : volatility-unceratin max-mean 
Running time:20.36
Balanced-Accuracy on testing set：91.16%
Recall on testing set：92.02%
F-measure on testing set：76.95%
Score:0.859


In [92]:
'''volatility-bias LR: model_in.coef_ is the estimator obtained by LR,[0.26,2.27] is the volatility uncertainty
obtained by cross-validition analysis
'''

y_preX_v=F_u([0.26,2.27],np.dot(x_te,model_in.coef_.T)+model_in.intercept_).reshape(x_te.shape[0],)

y_preX_v[y_preX_v<0.5]=0
y_preX_v[y_preX_v>=0.5]=1
print(classification_report(y_te,y_preX_v))
bacc=balanced_accuracy_score(y_te,y_preX_v)
fbeta=fbeta_score(y_te,y_preX_v,beta=2.94)
recall=recall_score(y_te,y_preX_v)
print('Classifier : volatility-unceratin LR ')
print("Running time:%.2f"%(end_i-start_i))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_preX_v)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_preX_v)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_preX_v,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_i-start_i)))



              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3812
           1       0.65      0.78      0.71       188

    accuracy                           0.97      4000
   macro avg       0.82      0.88      0.85      4000
weighted avg       0.97      0.97      0.97      4000

Classifier : volatility-unceratin LR 
Running time:3.26
Balanced-Accuracy on testing set：88.07%
Recall on testing set：78.19%
F-measure on testing set：76.63%
Score:0.805


In [86]:
s=time.time()
print(fsolve(lambda x:equa_m(x,145),0.5 ))
end=time.time()
print(end-s)

[4.19223182]
0.09584379196166992


In [98]:
'''mean-unceratin LR: model_in.coef_ is the estimator obtained by max-mean loss,4.26 is the upper mean
obtained by cross-validition analysis
'''
y_preX_m=sigmoid_((np.dot(x_te,model_in.coef_.T)+model_in.intercept_).reshape(x_te.shape[0],),4.26
                  
                 )

y_preX_m[y_preX_m<0.5]=0
y_preX_m[y_preX_m>=0.5]=1
print(classification_report(y_te,y_preX_m))
print('Classifier : mean-unceratin LR ')
print("Running time:%.2f"%(end_i-start_i))
bacc=balanced_accuracy_score(y_te,y_preX_m)
fbeta=fbeta_score(y_te,y_preX_m,beta=2.94)
recall=recall_score(y_te,y_preX_m)
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_preX_m)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_preX_m)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_preX_m,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_i-start_i)))


              precision    recall  f1-score   support

           0       1.00      0.89      0.94      3812
           1       0.29      0.93      0.44       188

    accuracy                           0.89      4000
   macro avg       0.64      0.91      0.69      4000
weighted avg       0.96      0.89      0.92      4000

Classifier : mean-unceratin LR 
Running time:3.21
Balanced-Accuracy on testing set：90.66%
Recall on testing set：92.55%
F-measure on testing set：75.35%
Score:0.856


In [64]:
'''mean-unceratin max-mean: w_2 is the estimator obtained by max-mean loss, 0.28 is the upper mean
obtained by cross-validition analysis
'''

y_preX_x=sigmoid_(np.dot(X_test,w_2).reshape(x_te.shape[0],),0.28)

y_preX_x[y_preX_x<0.5]=0
y_preX_x[y_preX_x>=0.5]=1
print(classification_report(y_te,y_preX_x))
bacc=balanced_accuracy_score(y_te,y_preX_x)
fbeta=fbeta_score(y_te,y_preX_x,beta=2.94)
recall=recall_score(y_te,y_preX_x)
print('Classifier : mean-unceratin max-mean ')
print("Running time:%.2f s"%(end_x-start_x))
# from sklearn.metrics import accuracy_score
print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_preX_x)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_preX_x)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_preX_x,beta=2.94)))
print('Score:{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_x-start_x)))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95      3812
           1       0.32      0.92      0.47       188

    accuracy                           0.90      4000
   macro avg       0.66      0.91      0.71      4000
weighted avg       0.96      0.90      0.92      4000

Classifier : mean-unceratin max-mean 
Running time:20.98 s
Balanced-Accuracy on testing set：91.13%
Recall on testing set：92.02%
F-measure on testing set：76.88%
Score:0.859
